#  Assign colors based on values of a property

While simple point visualization can show nicely the location of the point features one might want to inspect and discover various properties of 
points/objects/features in the map and use them to create specific visualizations.





## Imports

alter sys.path so we can import utils module

In [66]:
import sys
import os
# Add parent directory to sys.path
sys.path.insert(0, os.path.abspath('..'))

In [67]:
import utils

In [68]:
import leafmap.maplibregl as leafmap

## Data

We are going to vizualize a point layer representing building from east of Bangladesh. Geohub features [REST API](https://dev.undpgeohub.org/api)
and as a result it is possible to searh for data usingh the [/datasets](https://dev.undpgeohub.org/api/datasets) endpoint. The [utils module](../utils.py) contains a search function that takes a query expression or workd and returns a dictionary where the key is the dataset id and value is a list of links.

In [69]:
query='bangladesh buildings'
results = utils.search_datasets(query=query)

In [70]:

ds_id, links = next(iter(results.items()))
pmtiles_url = [e['href'] for e in links if e['rel'] == 'download'].pop()


## Visualization

leafmap is equipped with some util functions that allow us to inspect the PMTiles file

In [71]:
metadata = leafmap.pmtiles_metadata(pmtiles_url)
print(f"layer names: {metadata['layer_names']}")
print(f"bounds: {metadata['bounds']}")


layer names: ['bldgsc']
bounds: [90.8483229, 23.3422379, 91.063028, 23.510859]


## Attributes or properties

Vector tiles are very efficient with compressing attributes or properties. The medata variable crteated above can be used to explore/dicover
the properties of attributes that existy in the data file.


In [72]:
import json
#print(json.dumps(metadata, indent=2))
fields = metadata['vector_layers'][0]['fields']
print(json.dumps(fields, indent=4))
attrs = metadata['tilestats']['layers'][0]['attributes']
print('\n')


{
    "area_in_meters": "Number",
    "bf_source": "String",
    "boundary_id": "Mixed",
    "confidence": "Number",
    "country_iso": "String",
    "geohash": "String",
    "s2_id": "Mixed"
}




The best chance to assign a color to every point is to inspoect the values of a string type attribute or property like **bf_source**


In [73]:
print('\n')
for e in attrs:
    if e['attribute'] == 'bf_source':
        print(f'bf_source attribute values are {e["values"]}\n')




bf_source attribute values are ['google', 'microsoft', 'osm']



We will assign a scperific color to every points based on the values of the **bf_source** attribute

In [74]:
m = leafmap.Map(center=[0, 20], zoom=2, height="600px", 
                style='https://dev.undpgeohub.org/api/mapstyle/style.json'
               )

# create a maplibre style object as a python dictionary 
simple_point_style = {
    "version": 8,
    "sources": {
        "example_source": {
            "type": "vector",
            "url": "pmtiles://" + pmtiles_url,
            "attribution": "PMTiles",
        }
    },
    "layers": [
        {
            "id": "bbuildings",
            "source": "example_source",
            "source-layer": "bldgsc",
            "type": "symbol",
            "layout": {
                        "icon-image": "building",
                        "icon-size": 1,
                        "icon-overlap": "never",
                        "text-variable-anchor": [
                            "top",
                            "bottom",
                            "left",
                            "right"
                        ],
                        "text-radial-offset": 0.5,
                        "text-justify": "auto",
                        "text-font": [
                            "Proxima Nova Regular"
                        ],
                        "text-field": [
                                      "get",
                                      "bf_source"
                                    ],
                        "text-size": 13,
                        "text-max-width": 10,
                        "visibility": "visible"
                    },
            "paint": {
                        "icon-color": [
                                      "match",
                                      [
                                        "get",
                                        "bf_source"
                                      ],
                                      "google",
                                      "#7fc97f",
                                      "microsoft",
                                      "#c1242b",
                                      "osm",
                                      "#092d90",
                                      "#000000"
                                    ],
                        "icon-opacity": 1,
                        "text-halo-width": 5,
                        "text-color": "rgb(11 39 198)",
                        "icon-halo-color": "rgb(255,255,255)",
                        "icon-halo-width": 1
                    }
        }
    ]
}
m.add_pmtiles(
    pmtiles_url,
    style=simple_point_style,
    visible=True,
    opacity=1.0,
    tooltip=True,
)

legend_dict = {
        "google":"#7fc97f",
        "microsoft" : "#c1242b",
        "osm" : "#092d90"
}
m.add_legend(
    title="Building source provider",
    legend_dict=legend_dict,
    bg_color="rgba(255, 255, 255, 0.5)",
    position="bottom-left",
)
m.layer_interact()


In [75]:
m

Map(height='600px', map_options={'bearing': 0, 'center': (0, 20), 'pitch': 0, 'style': 'https://dev.undpgeohub…

In [76]:
m.fit_bounds(metadata['bounds'])